Let's see what the target formatting was:

In [32]:
import pandas as pd

psfile = "ParlaSpeech-HR.v1.0.jsonl"
ps = pd.read_json(psfile, lines=True, orient="records", nrows=10)
ps.head(2)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,speaker_info,split
0,seg.qNpeHxO0WzA_1967.6-1986.11.flac,"24 1 2020 - 16. sjednica, 9. saziv [qNpeHxO0Wz...",1967.60,1986.11,"[primjeni, željezničkih, paketa, kojima, se, p...","[0, 0.51, 0.99, 1.5, 1.82, 1.99, 2.99, 3.12, 3...","[primjeni, željezničkih, paketa, kojima, se, p...","[0, 0.51, 0.99, 1.5, 1.82, 1.99, 2.99, 3.12, 3...",ParlaMint-HR_S16.u1923,ParlaMint-HR_S16.u1924,None,None
1,seg.-k1z8behXXg_14758.42-14778.39.flac,"12 6 2019 - 12. sjednica, 9. saziv [-k1z8behXX...",14758.42,14778.39,"[km, i, distribuira, ih, na, nekih, 2600, prik...","[0, 0.25, 0.36, 0.93, 1.06, 1.28, 1.9, 3.23, 3...","[km, i, distribuira, ih, na, nekih, dvije, tis...","[0, 0.25, 0.36, 0.93, 1.06, 1.28, 1.9, 2.09, 2...",ParlaMint-HR_S12.u5004,ParlaMint-HR_S12.u5004,"{'Speaker_role': 'Regular', 'Speaker_type': 'M...",train


In [33]:
ps.columns

Index(['path', 'orig_file', 'start', 'end', 'words', 'word_start_times',
       'norm_words', 'norm_words_start_times', 'utterance_id_start',
       'utterance_id_end', 'speaker_info', 'split'],
      dtype='object')

In [34]:
jv = pd.read_json("026_JV_segments_splits.jsonl", lines=True)
# kaldi_transcript is normalized, asr_transcription is not
jv.head(1).T

,0
file,audio/8szK0RCfaqc.wav
segment_file,seg_audio/8szK0RCfaqc__362.0__380.6__.wav
start,362.0
end,380.6
asr_transcription,domaćih granica da postane regionalni festival...
kaldi_transcript,domaćih granica da postane regionalni festival...
raw_transcript__matched_on_kaldi,"domaćih granica, da postane regionalni festiva..."
raw_transcript__matched_on_asr,"domaćih granica, da postane regionalni festiva..."
guest_name,Emir Hadžihafizbegović
guest_description,glumac


In [35]:
print("Current: ", jv.columns, "\nTarget: ", ps.columns)

Current:  Index(['file', 'segment_file', 'start', 'end', 'asr_transcription',
       'kaldi_transcript', 'raw_transcript__matched_on_kaldi',
       'raw_transcript__matched_on_asr', 'guest_name', 'guest_description',
       'host', 'kaldi_words', 'kaldi_word_starts', 'kaldi_word_ends',
       'speaker_breakdown', 'average_distance', 'similarity_ratio',
       'num_words', 'length', 'similarity_kaldi_match_vs_asr_match',
       'similarity_kaldi_vs_asr', 'similarity_kaldi_match_vs_asr',
       'similarity_kaldi_vs_asr_match', 'is_acceptable', 'split'],
      dtype='object') 
Target:  Index(['path', 'orig_file', 'start', 'end', 'words', 'word_start_times',
       'norm_words', 'norm_words_start_times', 'utterance_id_start',
       'utterance_id_end', 'speaker_info', 'split'],
      dtype='object')


In [36]:
jv = jv.rename(columns={
    "file": "orig_file",
    "segment_file": "path",
    "kaldi_words": "words",
    "kaldi_word_starts": "word_start_times",
    "asr_transcription": "wav2vec_transcription"
})
jv["norm_words"] = jv.kaldi_transcript
jv["norm_words_start_times"] = jv.word_start_times

jv["utterance_id_start"] = None
jv["utterance_id_end"] = None

jv["speaker_info"] = jv.head().apply(lambda row: {
                        c: row[c] for c in ['guest_name', 'guest_description',
                        'host', 'speaker_breakdown']
                    }, axis=1)
jv["norm_words"] = jv.norm_words.str.split()
# jv["words"] = jv.words.str.split()


In [37]:
print("Current: ", jv.columns, "\nTarget: ", ps.columns)

Current:  Index(['orig_file', 'path', 'start', 'end', 'wav2vec_transcription',
       'kaldi_transcript', 'raw_transcript__matched_on_kaldi',
       'raw_transcript__matched_on_asr', 'guest_name', 'guest_description',
       'host', 'words', 'word_start_times', 'kaldi_word_ends',
       'speaker_breakdown', 'average_distance', 'similarity_ratio',
       'num_words', 'length', 'similarity_kaldi_match_vs_asr_match',
       'similarity_kaldi_vs_asr', 'similarity_kaldi_match_vs_asr',
       'similarity_kaldi_vs_asr_match', 'is_acceptable', 'split', 'norm_words',
       'norm_words_start_times', 'utterance_id_start', 'utterance_id_end',
       'speaker_info'],
      dtype='object') 
Target:  Index(['path', 'orig_file', 'start', 'end', 'words', 'word_start_times',
       'norm_words', 'norm_words_start_times', 'utterance_id_start',
       'utterance_id_end', 'speaker_info', 'split'],
      dtype='object')


In [38]:
set(ps.columns) - set(jv.columns)

set()

In [40]:
jv[ps.columns].head(2)

,path,orig_file,start,end,words,word_start_times,norm_words,norm_words_start_times,utterance_id_start,utterance_id_end,speaker_info,split
0,seg_audio/8szK0RCfaqc__362.0__380.6__.wav,audio/8szK0RCfaqc.wav,362.0,380.6,"[domaćih, granica, da, postane, regionalni, fe...","[362.11, 362.56, 363.02, 363.13, 363.51, 364.0...","[domaćih, granica, da, postane, regionalni, fe...","[362.11, 362.56, 363.02, 363.13, 363.51, 364.0...",None,None,"{'guest_name': 'Emir Hadžihafizbegović', 'gues...",train
1,seg_audio/F_Qt-sluUMM__368.7__377.0__.wav,audio/F_Qt-sluUMM.wav,368.7,377.0,"[ne, znam, da, li, je, neko, progovorio, na, t...","[368.85, 369.39, 369.57, 369.73, 370.08, 370.1...","[ne, znam, da, li, je, neko, progovorio, na, t...","[368.85, 369.39, 369.57, 369.73, 370.08, 370.1...",None,None,"{'guest_name': 'Miloš Grozdanović', 'guest_des...",train


In [48]:
jv[ps.columns].to_json("JuzneVesti-SR.v1.0.jsonl")